In [2]:
from __future__ import absolute_import

%matplotlib inline 
import numpy as np
import pylab
import os
from keras.layers import Layer, InputSpec
import matplotlib.pyplot as plt
import sys
import pandas as pd
import keras
import keras.backend.tensorflow_backend as K
from keras.layers import Input, LSTM, Dropout, Conv1D
from keras.models import Model,Sequential
from keras.layers.core import Dense, Lambda, Activation, Flatten
from keras import objectives
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import History
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping 
from statistics import mean, median
from sklearn.preprocessing import MinMaxScaler
from google.cloud import bigquery
K.set_session
plt.style.use('ggplot')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/Users/tatsuyahagiwara/d/bigquery.json'

In [3]:
"""
前処理
get_data: データを前処理して深層学習で扱える形に変換する
transform_data: データを正規化してLSTMで学習が容易な形に変更する。
"""

def get_data(data, time_steps):
    docX, docY = [], []
    for i in range(len(data)-time_steps):
        docX.append(data[i:i+time_steps])
        docY.append(data[i+time_steps])
    alsX = np.array(docX)
    alsY = np.array(docY)
    return alsX, alsY

def transform_data(data, inverse_option, scaler):
    data_shape = data.shape
    if inverse_option is True:
        data = scaler.inverse_transform(data)
    else:
        data = scaler.fit_transform(data)
    data = data.reshape(data_shape)
    return data, scaler

def prepare_data(original_data, time_steps):
    copy_data = original_data.copy()
    scaler = MinMaxScaler(feature_range=(0, 1), copy=False)
    data, scaler = transform_data(data=copy_data, 
                              inverse_option=False, scaler=scaler)
    
    x,y = get_data(data, time_steps=time_steps)
    return x, y, scaler  



# データのplot
def show_graph(day, data, label, color="b"):
    pylab.figure(figsize=(14, 8))
    pylab.subplot(211)
    pylab.xlabel('time')
    pylab.ylabel('temperature')
    pylab.plot(day, data, color=color, label=label)
    pylab.legend(loc='upper right')
    pylab.show()

# 学習データとその予測データのplot
def predict_model_show_graph(day, x, y_, scaler, model, time_steps):
    preds = model.predict(x)
    x = np.reshape(y_, (len(y_),))
    x_scale, scaler = transform_data(data=y_, inverse_option=True, scaler=scaler)
    predict_scale, scaler = transform_data(data=preds, inverse_option=True, scaler=scaler)
    mse_value = [(v - p_v)**2 for v, p_v in zip(x_scale[:,0], predict_scale[:,0])]
    return predict_scale, x_scale, mse_value

# 閾値計算
def calculate_mse(value, predict_value, variance=1.0):
    value = value[:, 0]
    predict_value = predict_value[:, 0]
    mse_value = [(v - p_v)**2 for v, p_v in zip(value, predict_value)]
    return np.array(mse_value)

# 閾値とデータplot
def show_graph_threshold(day, data, threshold, label, color="b", fix_threshold=True):
    pylab.figure(figsize=(14, 8))
    pylab.subplot(211)
    pylab.xlabel('time')
    pylab.ylabel('anomaly score')
    if fix_threshold is True:
        pylab.plot(day, np.repeat(threshold, day.shape[0]), 'k-', color = "k", ls = "dashed")
    else:
        pylab.plot(day, threshold, 'k-', color = "k", ls = "dashed")
    pylab.plot(day, data, "r", label='Anomaly Score test')
    pylab.legend(loc='upper right')
    pylab.show()


In [4]:
"""
モデルの作成と重みのロード
"""
time_steps = 12
window = time_steps

model = Sequential()
model.add(LSTM(300, input_shape=(time_steps, 1),
         stateful=False,return_sequences=True))
model.add(Flatten())
model.add(Dense(1, kernel_initializer='lecun_uniform'))
model.add(Activation("linear"))
model.load_weights('/Users/tatsuyahagiwara/d/cnt_label/week_dir/model.ep200.h5')
model.compile(loss="mean_squared_error", optimizer="adam",)

In [5]:
# BigQueryからSQLでデータ取ってくる
client = bigquery.Client(project='neon-opus-132123')
job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = False

In [6]:
# 一日分データ（2018/07/01~2018/07/30）30d
# 一週間分データ(2018/01/08~2018/02/26) 7w
# 一ヶ月分データ(2018/01/01~2018/07/01) 7m
query =  """
#standardSQL
select format_timestamp("%Y%m%d%H", time) AS hours,
original_keyword,
COUNT(original_keywords) AS cnt
from `search_logs.search_log_*`, UNNEST(original_keywords) as original_keyword
where original_keyword is not NULL
AND _TABLE_SUFFIX BETWEEN '20180301' AND '20180301'
GROUP BY hours, original_keyword
ORDER BY hours asc
  """

In [7]:
query_job = client.query( query, job_config=job_config)
results = query_job.result()

# hour, words, cntのDataFrameの作成

csv=[[i.hours, i.original_keyword, i.cnt] for i in list(results)]
csv=pd.DataFrame(csv)
listed=['hours','words','cnt']
csv.columns = listed
csv['hours']=pd.to_datetime(csv['hours'], format='%Y%m%d%H')

In [8]:
words=set(csv['words'])
words=list(words)
len(words)

34715

In [9]:
# ワードごとの時系列データ作成

slist = []
for word in words:
    group = csv[csv['words'] == word]
    if len(group)>=24:
        slist.append(group)
        
cnt_dic = {}
for v in slist:
    for hour, word,cnt in zip(v['hours'],v['words'],v['cnt']):
        cnt_dic.setdefault(word, []).append(cnt)
        
words_list=list(cnt_dic.keys())  
words=[i for i in words if i in words_list]


hour_dic = {}
for v in slist:
    for hour, word,cnt in zip(v['hours'],v['words'],v['cnt']):
        hour_dic.setdefault(word, []).append(hour)
        
        
cnt_df = pd.DataFrame.from_dict(cnt_dic, orient='index')
cnt_df=cnt_df.T  #applymap(lambda x: x*1000)


hour_df = pd.DataFrame.from_dict(hour_dic, orient='index')
hour_df=hour_df.T 

hour_listed=[]
cnt_listed=[]
for i in words:
    hour_listed.append([hour_df[i],i])
    cnt_listed.append([cnt_df[i],i])

In [10]:
# 抽出ワード数
len(words)

207

In [11]:
# 検知したバーストワードをlistに保存

csvs=[]
original_threshold = 270.2200 
train_data_mean=4.964
num_words=len(words)
for i in range(num_words):
    cnt=pd.DataFrame(list(cnt_listed[i][0])).dropna()
    hour=pd.DataFrame(list(hour_listed[i][0])).dropna()
    hours=np.array(hour[window:])
    wo=cnt_listed[i][1] # each word
    
    x_predict, y_ ,scaler = prepare_data(cnt, time_steps)
    predicted, x_scale, mse_value = predict_model_show_graph(hour[window:], x_predict, y_, scaler, model, time_steps)
    mse_value = calculate_mse(x_scale, predicted)
    
    N=int(np.mean(x_scale)/2)
    high_threshold = original_threshold*N  # 閾値
    if high_threshold<original_threshold:
        #print('high_threshold: {}'.format(original_threshold))
        #show_graph_threshold(hour[window:], mse_value, original_threshold, 'Anomaly Score test', "r")
        # 急上昇ワードの時間を表示
        for mse, hour in zip(mse_value, hours):
            if mse >=original_threshold:
                csvs.append([hour, wo])
    else:
        #print('high_threshold: {}'.format(high_threshold))
        #show_graph_threshold(hour[window:], mse_value, high_threshold, 'Anomaly Score test', "r")
        # 急上昇ワードの時間を表示
        for mse, hour in zip(mse_value, hours):
            if mse >=high_threshold:
                csvs.append([hour, wo])
    

In [12]:
df=pd.DataFrame(csvs, columns=['hour','words'])
df.shape

(1, 2)

In [14]:
# Nに日にちを指定
No='N'

# 対象ではない日のデータを削除
dff=[]
for i,v in df.iterrows():
    if str(v['hour'])[10:12] not in No:
        dff.append([v['hour'],v['words']])
        
dff=pd.DataFrame(df, columns=['hour','words'])

In [16]:
# 文字列に変換し、重複ワード削除
dff=dff.applymap(lambda x: str(x))
dff=dff.drop_duplicates(subset='words')

In [17]:
dff #バーストワードと日時

,hour,words
0,[2018-03-01T11:00:00.000000000],レディース
